In [47]:
import pandas as pd
import numpy as np

# Đọc dữ liệu từ file CSV
file_path = 'DL_TongHop.csv'  # Đường dẫn tới file CSV của bạn
df = pd.read_csv(file_path)

# Chuyển cột 'Thang' sang định dạng datetime và đặt làm chỉ mục
df['Thang'] = pd.to_datetime(df['Thang'], format='%Y-%m')
df.set_index('Thang', inplace=True)

# Tạo DataFrame rỗng để lưu trữ kết quả
df_interpolated = pd.DataFrame()

# Nội suy thêm một điểm giữa mỗi cặp điểm
for i in range(len(df) - 1):
    # Lấy hai điểm liên tiếp
    row1 = df.iloc[i]
    row2 = df.iloc[i + 1]
    
    # Tính giá trị trung bình của hai điểm
    interpolated_row = (row1 + row2) / 2
    
    # Tạo một timestamp ở giữa hai điểm gốc
    interpolated_time = row1.name + (row2.name - row1.name) / 2
    
    # Thêm giá trị vào DataFrame mới
    df_interpolated = df_interpolated.append(pd.DataFrame([row1, interpolated_row], index=[row1.name, interpolated_time]))
    
# Thêm điểm cuối cùng
df_interpolated = df_interpolated.append(df.iloc[-1])

# Tính phương sai của các cột dữ liệu gốc
original_variances = df.var()
print(original_variances)

# Tạo các giá trị nhiễu ngẫu nhiên từ phân phối chuẩn cho các cột nội suy
for column in df.columns:
    mu, sigma = 0, np.sqrt(original_variances[column])
    lower_limit = -0.2* sigma
    upper_limit = 0.2 * sigma
    
    # Tạo nhiễu theo phân phối chuẩn
    noise = np.random.normal(mu, sigma, df_interpolated.shape[0])
    
    # Giới hạn giá trị nhiễu trong khoảng giới hạn chuẩn đã tính toán
    noise = np.clip(noise, -lower_limit, upper_limit)
    
    # Chỉ thêm nhiễu vào các điểm nội suy
    noise[::2] = 0  # Giữ nguyên các giá trị gốc (các điểm chẵn)
    
    df_interpolated[column] += noise

print(df_interpolated)


SoLuong         2.429434e+10
SoLuongDT       1.374821e+12
TienKM          2.603619e+19
CongTrinhXD     1.346615e+06
CPI             1.825907e-01
XuatKhau        1.016413e+07
NhapKhau        7.637353e+06
SxCongNghiep    6.300352e+01
FDIDangKy       1.854262e+00
FDIGiaiNgan     2.686479e-01
BanLe           4.730649e+09
dtype: float64
                           SoLuong     SoLuongDT        TienKM  CongTrinhXD  \
2021-01-01 00:00:00  265547.000000  1.769949e+06  3.299285e+09  1291.666667   
2021-01-16 12:00:00  270588.782215  1.496796e+06  4.114789e+09  1523.754172   
2021-02-01 00:00:00  213284.000000  7.546320e+05  2.889267e+09  1291.666667   
2021-02-15 00:00:00  457274.782215  1.872166e+06  4.152122e+09  1523.754172   
2021-03-01 00:00:00  638919.000000  2.520689e+06  3.373951e+09  1291.666667   
...                            ...           ...           ...          ...   
2023-10-01 00:00:00  264714.000000  3.428062e+06  5.011626e+09  1009.000000   
2023-10-16 12:00:00  396623.282215

C:\Users\User\AppData\Local\Temp\ipykernel_7464\3353500484.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_interpolated = df_interpolated.append(pd.DataFrame([row1, interpolated_row], index=[row1.name, interpolated_time]))
C:\Users\User\AppData\Local\Temp\ipykernel_7464\3353500484.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_interpolated = df_interpolated.append(pd.DataFrame([row1, interpolated_row], index=[row1.name, interpolated_time]))
C:\Users\User\AppData\Local\Temp\ipykernel_7464\3353500484.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_interpolated = df_interpolated.append(pd.DataFrame([row1, interpolated_row], index=[row1.name, interpolated_time]))
C:\Users\User\AppData\Local\Temp\ipykern

In [45]:
df_interpolated.to_csv('da_ta.csv')